## Includes

In [1]:
import sys
import os
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config


config_info = {
  "canalyst_api_key": "",
   "s3_access_key_id": "",
    "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}

config = Config(config=config_info)

A configuration file has been created for you in 
/Users/jedgore/canalyst/keys.json.


# Example: Redrive One Model
## Workflow:
<li>Create ModelSet for one ticker
<li>Create / show ModelMap
<li>Get Key Revenue Drivers
<li>Show the forecast dataset for Key Drivers
<li>Make an adjustment of one parameter
<li>Redrive model and display new values vs old

## Change ticker as needed

In [2]:
ticker = "AXP US"
return_series = "MO_RIS_REV" 
#return series variable is used in the model map end node AND the return value from fit()

## Create ModelSet

In [3]:
cd_modelset = cd.ModelSet(ticker_list=[ticker],config=config)

## Create ModelMap and show() it

In [4]:
time_series_name=return_series
model_map = cd_modelset.create_model_map(ticker=ticker,
                            time_series_name=time_series_name,
                            tree = True,
                            notebook = True,
                            col_for_labels = "time_series_description") #change to time_series_name to see range names
model_map.show()

https://mds.canalyst.com/api/equity-model-series/FYXPQQ0114/equity-models/FY2021.22/time-series/MO_RIS_REV/forecast-data-points/Q1-2022/drivers/?format=dot


## Get the Key Drivers list from ModelMap

In [5]:
key_drivers = list(set(list(cd_modelset.models[ticker].key_driver_map(return_series)['time_series_name'])))
key_drivers

https://mds.canalyst.com/api/equity-model-series/FYXPQQ0114/equity-models/FY2021.22/time-series/MO_RIS_REV/forecast-data-points/Q1-2022/drivers/?format=dot


['z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth',
 'z_FYXPQQ0114_MO_OS_OtherFeesandCommissionsasofWorldwideProprietaryCardBilledBusiness',
 'MO_GA_TPV',
 'z_FYXPQQ0114_MO_OS_proprietarycardsasoftotalcardsinforcecalculated',
 'z_FYXPQQ0114_MO_OS_Otherrevenuegrowth',
 'MO_KPI_TakeRate',
 'MO_NII_AssetYield',
 'z_FYXPQQ0114_MO_OS_ofbilledbusinessfromproprietarycardscalculated',
 'z_FYXPQQ0114_MO_OS_averageloansincludinghfsloanportfoliosgrowth',
 'z_FYXPQQ0114_MO_OS_YYchangeinAverageFeePerCard']

## Create a view of forecasted Key Drivers

In [6]:
cd_modelset.model_frame(time_series_name=key_drivers,
                       is_historical=False,
                       period_duration_type="fiscal_quarter",
                       pivot=True)

period_name_sorted,ticker,name_index,category,time_series_name,time_series_description,is_driver,MRFQ,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4
0,AXP US,10,Growth Analysis,z_FYXPQQ0114_MO_OS_Otherrevenuegrowth,"Other revenue growth, %",True,Q4-2021,20.00,20.00,10.00,10.00,0.00,20.00,10.00,10.00
1,AXP US,15,Growth Analysis,MO_GA_TPV,"Worldwide Network volume Growth, %",True,Q4-2021,35.00,20.00,20.00,15.00,5.00,7.00,7.00,7.00
2,AXP US,16,Growth Analysis,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",True,Q4-2021,9.00,5.00,5.00,5.00,1.00,1.00,1.00,1.00
3,AXP US,17,Growth Analysis,z_FYXPQQ0114_MO_OS_averageloansincludinghfsloanportfoliosgrowth,"Total Loans Including HFS Loan Portfolios Growth, %",True,Q4-2021,15.00,8.50,8.50,8.50,15.00,8.50,8.50,8.50
4,AXP US,35,Operating Stats - Discount Revenue (FS),z_FYXPQQ0114_MO_OS_ofbilledbusinessfromproprietarycardscalculated,"% of Billed business from proprietary cards (calculated), %",True,Q4-2021,88.30,88.30,88.30,88.30,88.30,88.30,88.30,88.30
5,AXP US,37,Operating Stats - Discount Revenue (FS),MO_KPI_TakeRate,"Average Discount Rate, %",True,Q4-2021,2.26,2.26,2.26,2.26,2.26,2.26,2.26,2.26
6,AXP US,49,Operating Stats - Net Card Fees (FS),z_FYXPQQ0114_MO_OS_proprietarycardsasoftotalcardsinforcecalculated,"Proprietary cards as % of total cards-in-force (calculated), %",True,Q4-2021,61.30,61.30,61.30,61.30,61.30,61.30,61.30,61.30
7,AXP US,52,Operating Stats - Net Card Fees (FS),z_FYXPQQ0114_MO_OS_YYchangeinAverageFeePerCard,"Change in Average Fee Per Card, %",True,Q4-2021,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
8,AXP US,57,Operating Stats - Other Fees and Commissions (FS),z_FYXPQQ0114_MO_OS_OtherFeesandCommissionsasofWorldwideProprietaryCardBilledBusiness,"Other Fees and Commissions as % of Worldwide Proprietary Card Billed Business, %",True,Q4-2021,0.22,0.25,0.25,0.25,0.25,0.25,0.25,0.25
9,AXP US,67,Operating Stats - Net Interest Income (FS),MO_NII_AssetYield,"Net Interest Income Divided By Average Loans, %",True,Q4-2021,10.20,10.20,10.20,10.20,10.20,10.20,10.20,10.20


## Get the number of forecast periods for use in forecast_frame method

In [7]:
forecast_periods = cd_modelset.model_frame(time_series_name=key_drivers,
                       is_historical=False,
                       period_duration_type="fiscal_quarter",
                       pivot=False).groupby('period_name').first().reset_index().shape[0]

## Multiple one Key Driver (here I picked the first) by 1% across the forecast period

In [10]:
time_series_name = key_drivers[0]
print(time_series_name)
df_1_param = cd_modelset.forecast_frame(time_series_name,
                         n_periods=forecast_periods,
                         function_name='multiply',
                         function_value=1.01)

df_1_param #display the param dataframe which is used in fit()

z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth


,ticker,period_name,time_series_name,time_series_description,value,new_value,period_end_date
8,AXP US,Q1-2022,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",9.00000,9.09000,2022-03-31
465,AXP US,Q2-2022,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",5.00000,5.05000,2022-06-30
922,AXP US,Q3-2022,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",5.00000,5.05000,2022-09-30
1379,AXP US,Q4-2022,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",5.00000,5.05000,2022-12-31
1836,AXP US,Q1-2023,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",1.00000,1.01000,2023-03-31
2293,AXP US,Q2-2023,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",1.00000,1.01000,2023-06-30
2750,AXP US,Q3-2023,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",1.00000,1.01000,2023-09-30
3207,AXP US,Q4-2023,z_FYXPQQ0114_MO_OS_totalcardsinforcegrowth,"Total Cards-in-Force Growth, %",1.00000,1.01000,2023-12-31


## Run the ModelSet.fit() function to re drive the model
### "Default" is the model current default, "Scenario" shows our changes

In [9]:
df_output= cd_modelset.fit(df_1_param,return_series) #our fit function will return a link to scenario engine JSON for audit
df_output = cd_modelset.filter_summary(df_output,period_type='Q')
df_output

AXP US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/FYXPQQ0114/equity-models/FY2021.22/scenarios/0DphpTTjX1yEhdT8drth-A/


,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,AXP US,Q1-2022,MO_RIS_REV,Net Revenue,11764878714.16521,11765465252.2669,1.00005
1,AXP US,Q2-2022,MO_RIS_REV,Net Revenue,12207512647.51370,12208445949.5175,1.00008
2,AXP US,Q3-2022,MO_RIS_REV,Net Revenue,12734658246.87628,12735356148.3483,1.00005
3,AXP US,Q4-2022,MO_RIS_REV,Net Revenue,13790913822.38791,13791635278.7796,1.00005
4,AXP US,Q1-2023,MO_RIS_REV,Net Revenue,12582382361.95676,12583417561.3861,1.00008
5,AXP US,Q2-2023,MO_RIS_REV,Net Revenue,13142294932.93300,13143402553.3591,1.00008
6,AXP US,Q3-2023,MO_RIS_REV,Net Revenue,13610264215.27972,13611132883.2102,1.00006
7,AXP US,Q4-2023,MO_RIS_REV,Net Revenue,14748377233.05105,14749275219.4515,1.00006
